In [1]:
from __future__ import print_function
import pandas as pd
import girder_client, os,sys, random, pprint
import numpy as np
import colour as color

from masks_to_annotations_handler_simple import (
     get_annotation_documents_from_contours)

In [2]:
##color.RGB_TO_COLOR_NAMES contains some named colors  ## This is just a random colorSet I am using as I load the shapes
colorSet = []

for (r,g,b) in color.RGB_TO_COLOR_NAMES.keys():
    colorSet.append("rgb(%s,%s,%s)" % (r,g,b))

In [3]:
gc = girder_client.GirderClient(apiUrl='https://styx.neurology.emory.edu/girder/api/v1')
gc.authenticate(interactive=True)

Login or email: admin
Password for admin: ········


{'_accessLevel': 2,
 '_id': '5cacfa86e62914004a60e139',
 '_modelType': 'user',
 'admin': True,
 'created': '2019-04-09T20:03:18.294000+00:00',
 'email': 'histomicstk.admin@kitware.com',
 'emailVerified': True,
 'firstName': 'admin',
 'groupInvites': [],
 'groups': [],
 'lastName': 'admin',
 'login': 'admin',
 'otp': False,
 'public': True,
 'size': 0,
 'status': 'enabled'}

In [4]:
sampleItem ='5d9f35971e81085774dae6ac'
### This is a lung 40X image I am using
###https://styx.neurology.emory.edu/girder/#item/5d9f35971e81085774dae6ac
slideInfo = gc.get("/item/%s/tiles" %sampleItem)

In [5]:
maxX = slideInfo['sizeX']
maxY = slideInfo['sizeY']
tileSize = ts = 1024
xCoords =np.arange(0,maxX,tileSize)
yCoords =np.arange(0,maxY,tileSize)

In [11]:
gridData = []

for x in xCoords:
    for y in yCoords:
        ### need to make sure the Box doesn't extend past the end of the maxY
        ### not sure if/how this effects the coordinates
        xEnd = x+ts
        yEnd = y+ts
        
        if (xEnd > maxX):
            xEnd = maxX
        
        if (yEnd > maxY):
            yEnd = maxY
        
        
        boxInfo = {'xStart':x,
                   'xEnd':xEnd,
                   'yStart':y,
                   'yEnd': yEnd,
                    'objId':"%s-%s"% (x,y),
                    'coords_x' : "%s,%s,%s,%s,%s"% (x,xEnd,xEnd,x,x),
                    'coords_y' : "%s,%s,%s,%s,%s"% (y,y,yEnd,yEnd,y),
                   'group': "grid-%s"%ts,
                   'color': random.choice(colorSet),
                   'extra': {"Features":"TBD", "ADD":"MORE"}
                  }
        gridData.append(boxInfo)

In [12]:
#THIS IS THE IMPORTANT PART!!
contours_df = pd.DataFrame.from_records(gridData)

%load_ext autoreload
%autoreload 2
#10 is small.. it can be... he has tested 1000, can do more

#X_OFFSET would be 0

X_OFFSET = Y_OFFSET = 0

# get list of annotation documents
annprops = {
    'X_OFFSET': X_OFFSET,
    'Y_OFFSET': Y_OFFSET,
    'opacity': 0.0,  ## this should be 0 because that means it will only load if I Want to
    'lineWidth': 1.0,
}
annotation_docs = get_annotation_documents_from_contours(
    contours_df.copy(), separate_docs_by_group=False, annots_per_doc=10000,
    docnamePrefix='demoGrid_Ftrs_%s'%ts, annprops=annprops,
    verbose=True, monitorPrefix="" + ": annotation docs")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
: annotation docs: default: doc 1 of 1: contour 1 of 210
: annotation docs: default: doc 1 of 1: contour 2 of 210
: annotation docs: default: doc 1 of 1: contour 3 of 210
: annotation docs: default: doc 1 of 1: contour 4 of 210
: annotation docs: default: doc 1 of 1: contour 5 of 210
: annotation docs: default: doc 1 of 1: contour 6 of 210
: annotation docs: default: doc 1 of 1: contour 7 of 210
: annotation docs: default: doc 1 of 1: contour 8 of 210
: annotation docs: default: doc 1 of 1: contour 9 of 210
: annotation docs: default: doc 1 of 1: contour 10 of 210
: annotation docs: default: doc 1 of 1: contour 11 of 210
: annotation docs: default: doc 1 of 1: contour 12 of 210
: annotation docs: default: doc 1 of 1: contour 13 of 210
: annotation docs: default: doc 1 of 1: contour 14 of 210
: annotation docs: default: doc 1 of 1: contour 15 of 210
: annotation docs: default: doc 1 of 1: contour 16 

In [13]:
for annotation_doc in annotation_docs:
    resp = gc.post(
        "/annotation?itemId=" + sampleItem, json=annotation_doc)

In [14]:
## This will delete all annotations associated with an Item
## Use at your own risk!

curAnnotationsForItem = gc.get("/annotation?itemId=%s&limit=1000" % sampleItem)
print(len(curAnnotationsForItem))
for cafi in curAnnotationsForItem: ### this was only getting 50 elements per call
    pprint.pprint(cafi)
#     if 'demo' in cafi['annotation']['name']:
#         gc.delete("/annotation/%s" % cafi['_id'])

4
{'_accessLevel': 2,
 '_id': '5db5f72fb498bd9d29ad9a0c',
 '_modelType': 'annotation',
 '_version': 230269,
 'annotation': {'description': '', 'name': 'demoGrid_1024_default-0'},
 'created': '2019-10-27T19:59:43.586000+00:00',
 'creatorId': '5cacfa86e62914004a60e139',
 'groups': ['grid-1024'],
 'itemId': '5d9f35971e81085774dae6ac',
 'public': False,
 'updated': '2019-10-27T19:59:43.586000+00:00',
 'updatedId': '5cacfa86e62914004a60e139'}
{'_accessLevel': 2,
 '_id': '5db71f25b498bd9d29ae2873',
 '_modelType': 'annotation',
 '_version': 230767,
 'annotation': {'description': '', 'name': 'demoGrid_1024_default-0'},
 'created': '2019-10-28T17:02:28.984000+00:00',
 'creatorId': '5cacfa86e62914004a60e139',
 'groups': ['grid-1024'],
 'itemId': '5d9f35971e81085774dae6ac',
 'public': False,
 'updated': '2019-10-28T17:02:28.984000+00:00',
 'updatedId': '5cacfa86e62914004a60e139'}
{'_accessLevel': 2,
 '_id': '5db8a3a80513d06ecb050ff1',
 '_modelType': 'annotation',
 '_version': 231049,
 'annotation

In [ ]:
## group tileGrid512
## color rgb(r,g,b) .. literally a sa string
## has_holes I can ignore
## ignore the touches_edge properties
## coords_x and coords_y is

# {'description': '',
#  'elements': [{'closed': True,
#    'fillColor': 'rgba(0,255,0,0.2)',
#    'group': 'normal_acinus_or_duct',
#    'label': {'value': 'normal_acinus_or_duct'},
#    'lineColor': 'rgb(0,255,0)',
#    'lineWidth': 4.0,
#    'points': [[62081.0, 36019.0, 0.0],
#     [62081.0, 36021.0, 0.0],
#     [62080.0, 36022.0, 0.0],
#     [62080.0, 36025.0, 0.0],
#     [62079.0, 36026.0, 0.0]],
#    'type': 'polyline'},
#   {'closed': True,
#    'fillColor': 'rgba(0,255,0,0.2)',
#    'group': 'normal_acinus_or_duct',
#    'label': {'value': 'normal_acinus_or_duct'},
#    'lineColor': 'rgb(0,255,0)',
#    'lineWidth': 4.0,
#    'points': [[61743.0, 35806.0, 0.0],
#     [61742.0, 35807.0, 0.0],
#     [61741.0, 35807.0, 0.0],
#     [61740.0, 35808.0, 0.0],
#     [61738.0, 35808.0, 0.0]],
#    'type': 'polyline'}],
#  'name': 'demo_normal_acinus_or_duct-0'}


In [16]:
gc.get("/annotation/5db8a4c00513d06ecb05134d")  ## if I want to get a specific annotation document

{'_accessLevel': 2,
 '_id': '5db8a4c00513d06ecb05134d',
 '_modelType': 'annotation',
 '_version': 231056,
 'annotation': {'description': '',
  'elements': [{'group': 'grid-1024',
    'lineColor': 'rgb(0,250,154)',
    'extra': {'ADD': 'MORE', 'Features': 'TBD'},
    'label': {'value': 'grid-1024'},
    'points': [[0.0, 0.0, 0.0],
     [1024.0, 0.0, 0.0],
     [1024.0, 1024.0, 0.0],
     [0.0, 1024.0, 0.0],
     [0.0, 0.0, 0.0],
     [0.0, 0.0, 0.0]],
    'closed': True,
    'lineWidth': 1.0,
    'type': 'polyline',
    'id': '5db8a4c00513d06ecb05134e'},
   {'group': 'grid-1024',
    'lineColor': 'rgb(132,112,255)',
    'extra': {'ADD': 'MORE', 'Features': 'TBD'},
    'label': {'value': 'grid-1024'},
    'points': [[0.0, 1024.0, 0.0],
     [1024.0, 1024.0, 0.0],
     [1024.0, 2048.0, 0.0],
     [0.0, 2048.0, 0.0],
     [0.0, 1024.0, 0.0],
     [0.0, 1024.0, 0.0]],
    'closed': True,
    'lineWidth': 1.0,
    'type': 'polyline',
    'id': '5db8a4c00513d06ecb05134f'},
   {'group': 'grid-